#1.定义学习率
#2.构建transforms，对图像做变换
#3.下载，加载数据集MNIST
#4.构建网络模型
#5.定义训练方法
#6.定义测试方法
#7.开始训练模型，输出预测结果

In [1]:
#加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
#定义超参数
BATCH_SIZE = 128 # 每批次处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #是否用gpu训练
EPOCHS = 20 #训练数据集的轮次

In [3]:
#构建pipeline，处理图像
pipeline = transforms.Compose([
    transforms.ToTensor(),#将图片转换为Tensor
    transforms.Normalize((0.1307,),(0.3081, ))#标准化，降低复杂度
])

In [4]:
#加载数据
from torch.utils.data import DataLoader
#下载
train_set = datasets.MNIST("data",train=True, download=True, transform=pipeline) #训练集

test_set = datasets.MNIST("data",train=False, download=True, transform=pipeline) #测试集

#加载数据
train_loader = DataLoader(train_set , batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set , batch_size=BATCH_SIZE, shuffle=True)


In [5]:
#显示图片
with open("./data/MNIST/raw/t10k-images-idx3-ubyte","rb") as f:
    file = f.read()

In [6]:
imagel = [int(str(item).encode('ascii'),16)for item in file[16:16+784]] #用来保存图片

In [7]:
import cv2
import numpy as np

imagel_np = np.array(imagel,dtype=np.uint8).reshape(28,28,1)
print(imagel_np.shape)

(28, 28, 1)


In [8]:
cv2.imwrite("digit.jpg",imagel_np)

True

In [9]:
#构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5) # 1灰度图片的通道 10输出通道 5：kernel卷积核
        self.conv2 = nn.Conv2d(10,20,3) #10:输入通道 20：输出通道 3：kernel
        #全连接层
        self.fc1 =nn.Linear(20*10*10,500) #20*10*10:输入通道，500：输出通道
        self.fc2 = nn.Linear(500,10)#500:输入通道，10：输出通道

    def forward(self,x):
        input_size = x.size(0) #batch_size x 1(灰度） x 28 x 28（像素）
        x = self.conv1(x) #输入：batch*1*28*28，输出：batch*10*24*24（28-5+1=24）
        x = F.relu(x)#激活函数 保持形状不变 输出：batch*10*24*24
        x = F.max_pool2d(x,2,2)#池化层 ,把x变成分块矩阵，找其中的最大值（提取图片最明显的特征，忽略小细节）
                                #输入batch*10*24*24 输出：batch*10*12*12 （步长为2，减半）
        x = self.conv2(x) #输入batch*10*12*12 输出：batch*20*10*10（12-3+1=10）
        x = F.relu(x)
        
        x = x.view(input_size,-1)# 拉平，-1自动计算维度，20*10*10=2000 
        x = self.fc1(x) #输入：batch*2000 输出：batch*500（全连接层定义500）
        x = F.relu(x) # 保持shape不变

        x = self.fc2(x) #输入：Batch*500 输出：batch*10

        output = F.log_softmax(x,dim=1) #计算分类后，每个数字的概率值
        return output

In [10]:
#定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())#优化器，用adam每个训练参数的学习率可以自适应变化

In [11]:
#定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    #模型训练
    model.train()
    for batch_index,(data,target) in enumerate(train_loader):
        #部署到device上
        data, target = data.to(device), target.to(device)
        #梯度初始化
        optimizer.zero_grad()
        #预测 训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output,target)#交叉熵损失，针对多分类任务
        #反向传播 更新权重，神经元参数
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 ==0:
            print("Train Epoch:{}\t Loss:{:.6f}\n".format(epoch,loss.item()))
        
        

In [12]:
#定义测试方法
def test_model(model, device, test_loader):
    #模型验证
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad(): #不会计算梯度，也不进行反向传播
        for data, target in test_loader:
            #部署
            data,target = data.to(device), target.to(device)
            #测试数据
            output = model(data)
            #计算损失
            test_loss += F.cross_entropy(output,target).item()
            #找出最大值下标
            pred = output.argmax(dim=1)
            #累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        accuracy = 100.0*correct / len(test_loader.dataset)
        print("Test_Average Loss:{:.4f}\t Accuracy:{:.3f}%\n".format(test_loss,accuracy))

In [ ]:
#调用方法，输出结果
for epoch in range(1,EPOCHS+1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_result = test_model(model,DEVICE,test_loader)

Train Epoch:1	 Loss:2.303189

Test_Average Loss:0.0009	 Accuracy:98.250%

Train Epoch:2	 Loss:0.019744

Test_Average Loss:0.0008	 Accuracy:98.410%

Train Epoch:3	 Loss:0.010082

Test_Average Loss:0.0007	 Accuracy:98.560%

Train Epoch:4	 Loss:0.063900

Test_Average Loss:0.0007	 Accuracy:98.610%

Train Epoch:5	 Loss:0.011997

Test_Average Loss:0.0005	 Accuracy:98.930%

Train Epoch:6	 Loss:0.000366

Test_Average Loss:0.0008	 Accuracy:98.660%

Train Epoch:7	 Loss:0.000621

Test_Average Loss:0.0009	 Accuracy:98.700%

Train Epoch:8	 Loss:0.009907

Test_Average Loss:0.0006	 Accuracy:99.010%

Train Epoch:9	 Loss:0.019513

Test_Average Loss:0.0007	 Accuracy:98.940%

Train Epoch:10	 Loss:0.020159

Test_Average Loss:0.0007	 Accuracy:98.980%

Train Epoch:11	 Loss:0.034819

Test_Average Loss:0.0007	 Accuracy:99.000%

Train Epoch:12	 Loss:0.000521

Test_Average Loss:0.0011	 Accuracy:98.740%

Train Epoch:13	 Loss:0.000002

Test_Average Loss:0.0008	 Accuracy:98.980%

Train Epoch:14	 Loss:0.000102

Tes